# Урок 2. Масштабирование признаков. Регуляризация. Стохастический градиентный спуск.

## Домашнее задание:

### __1.__ Сгенерировать датасет при помощи `sklearn.datasets.make_regression` и обучить линейную модель при помощи градиентного спуска и стохастического градиентного спуска. Построить графики среднеквадратичной ошибки от числа итераций для обоих методов на одном рисунке, сделать выводы о разнице скорости сходимости каждого из методов.<br>
__Решение:__

__Импорты:__

In [1]:
from sklearn import datasets
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

__Создание датасета:__

In [2]:
data, target, coef = datasets.make_regression(n_samples=10000,
                                              n_features = 2,
                                              n_informative = 2,
                                              n_targets = 1, 
                                              noise = 5,
                                              coef = True,
                                              random_state = 42)

In [65]:
print(data)
print(target)
print(coef)

[[-0.58090755  1.56929749]
 [ 0.73205833  0.48200215]
 [ 0.19421711  1.09046879]
 ...
 [-1.90691676  0.13300016]
 [-1.03070079 -0.98595043]
 [ 1.15759762  0.53700316]]
[ -49.6854239    73.63062887   20.15499442 ... -192.63317609 -106.45952324
  118.37024675]
[98.57676058  5.56663928]


__Подготовка датасета:__

In [3]:
# Получим средние значения и стандартное отклонение по столбцам
 
means = np.mean(data, axis=0)
stds = np.std(data, axis=0)
# параметр axis указывается для вычисления значений по столбцам, а не по всему массиву
#(см. документацию в разделе источников)
 
# вычтем каждое значение признака из среднего и поделим на стандартное отклонение
for i in range(data.shape[0]):
    for j in range(data.shape[1]):
        data[i][j] = (data[i][j] - means[j])/stds[j]

In [4]:
# реализуем функцию, определяющую среднеквадратичную ошибку
def mserror(X, w, y_pred):
    y = X.dot(w)
    return (sum((y - y_pred)**2)) / len(y)

__Реализация SGD:__

In [63]:
%%time
# инициализируем начальный вектор весов
w = np.zeros(2)
 
# список векторов весов после каждой итерации
w_list = [w.copy()]
 
# список значений ошибок после каждой итерации
errors = []
 
# шаг градиентного спуска
eta = 0.5
 
# максимальное число итераций
max_iter = 1e5
 
# критерий сходимости (разница весов, при которой алгоритм останавливается)
min_weight_dist = 1e-8
 
# зададим начальную разницу весов большим числом
weight_dist = np.inf
 
# счетчик итераций
iter_num = 0
 
np.random.seed(1234)
 
# ход градиентного спуска
while weight_dist > min_weight_dist and iter_num < max_iter:
    
    # генерируем случайный индекс объекта выборки
    train_ind = np.random.randint(data.shape[0])
    
    new_w = w - 2 * eta * np.dot(data[train_ind].T, (np.dot(data[train_ind], w) - target[train_ind])) / target.shape[0]
 
    weight_dist = np.linalg.norm(new_w - w, ord=2)
    
    w_list.append(new_w.copy())
    errors.append(mserror(data, new_w, target))
    
    iter_num += 1
    w = new_w
    
w_list = np.array(w_list)
 
print(f'В случае использования стохастического градиентного спуска ошибка MSE составляет: {round(errors[-1], 4)}')
print(f'Количество итераций: {iter_num}')

В случае использования стохастического градиентного спуска ошибка MSE составляет: 25.0986
Количество итераций: 100000
Wall time: 1min 49s


In [66]:
print(w_list)

[[ 0.00000000e+00  0.00000000e+00]
 [-9.55718336e-06  9.45923508e-05]
 [ 2.65407960e-02  6.88828167e-03]
 ...
 [ 9.79328550e+01  5.63416411e+00]
 [ 9.79325807e+01  5.63359087e+00]
 [ 9.79324822e+01  5.63353712e+00]]


__Реализация GD:__

In [30]:
# реализуем функцию, определяющую среднеквадратичную ошибку
def calc_mse(y, y_pred):
    error = np.mean((y - y_pred)**2)
    return error

In [79]:
# реализуем функцию осуществляющую перебор параметров
X = np.reshape(data, (2, 10000))
y = target
def gradient_descent(alpha, iterations, W, X=X, y=y):
    n = X.shape[1]
    err = np.inf
    for i in range(iterations):
        y_pred = np.dot(W, X)
        err_new = calc_mse(y, y_pred)
        if err_new < err:
            err = err_new
            for j in range(W.shape[0]):
                W[j] -= alpha * (1/n * 2 * np.sum(X[j] * (y_pred - y))) 
        else:
            return i-1, W, err
    return i, W, err

In [51]:
# выведем результат предсказаний
y_pred = np.dot(W, X)
print(y)

[ -49.6854239    73.63062887   20.15499442 ... -192.63317609 -106.45952324
  118.37024675]


In [80]:
# подбор оптимальных параметров
min_err = np.inf
min_coef = ()

for alpha in [1e-1, 1e-2, 1e-3, 1e-4]:
    for iters in [1e3, 1e4, 1e5, 1e6]:
        W = np.array([0.0, 0.0])
        num, wi, err = gradient_descent(alpha, int(iters), W)
        print(f'Альфа = {alpha}, количество итераций = {iters}. Ошибка: {err}')
        if err < min_err:
            min_err = err
            min_coef = (alpha, iters)

print(f'\nМинимальная ошибка {min_err} достигается при коэфициентах {min_coef}')

Альфа = 0.1, количество итераций = 1000.0. Ошибка: 9641.707321992331
Альфа = 0.1, количество итераций = 10000.0. Ошибка: 9641.707321992331
Альфа = 0.1, количество итераций = 100000.0. Ошибка: 9641.707321992331
Альфа = 0.1, количество итераций = 1000000.0. Ошибка: 9641.707321992331
Альфа = 0.01, количество итераций = 1000.0. Ошибка: 9641.707321992395
Альфа = 0.01, количество итераций = 10000.0. Ошибка: 9641.707321992395
Альфа = 0.01, количество итераций = 100000.0. Ошибка: 9641.707321992395
Альфа = 0.01, количество итераций = 1000000.0. Ошибка: 9641.707321992395
Альфа = 0.001, количество итераций = 1000.0. Ошибка: 9641.718917546386
Альфа = 0.001, количество итераций = 10000.0. Ошибка: 9641.707321993226
Альфа = 0.001, количество итераций = 100000.0. Ошибка: 9641.707321993226
Альфа = 0.001, количество итераций = 1000000.0. Ошибка: 9641.707321993226
Альфа = 0.0001, количество итераций = 1000.0. Ошибка: 9642.14676990166
Альфа = 0.0001, количество итераций = 10000.0. Ошибка: 9641.71891796256

In [86]:
X = np.reshape(data, (2, 10000))
y = target
n = X.shape[1]
alpha = 1e-2
W = np.array([1.0, 1.0])
print(f'Number of objects = {n} \
       \nLearning rate = {alpha} \
       \nInitial weights = {W} \n')
print(X)

e = 0.0000001
last_err = None

for i in range(1000):
    y_pred = np.dot(W, X)
    err = calc_mse(y, y_pred)
    W -= alpha * (1/n * 2 * np.sum(X * (y_pred - y),  axis=1))
    
    if last_err == None:
        last_err = err
        continue

    elif i % 100 == 0:
        print(f'Итераций = {i}, Подобранные веса = {W}, Ошибка = {err}')
    
    elif last_err - err <= e:
        print(f'Вывод:\n Необходимое кол-во итераций = {i}\n Подобранные веса = {W}\n Финальная ошибка на данной итерации = {err}\n погрешность = {e}\n')
        break

    last_err = err

Number of objects = 10000        
Learning rate = 0.01        
Initial weights = [1. 1.] 

[[-0.58090755  1.56929749  0.73205833 ...  1.23497108  0.84429619
   1.22368979]
 [-0.45764694 -1.04164435  1.46297603 ... -0.98595043  1.15759762
   0.53700316]]
Итераций = 100, Подобранные веса = [-0.16503001  0.76412007], Ошибка = 9641.741836642303
Итераций = 200, Подобранные веса = [-0.32053703  0.73417722], Ошибка = 9641.707963696068
Итераций = 300, Подобранные веса = [-0.34177699  0.73028443], Ошибка = 9641.707333926686
Вывод:
 Необходимое кол-во итераций = 340
 Подобранные веса = [-0.34362165  0.72995982]
 Финальная ошибка на данной итерации = 9641.70732441681
 погрешность = 1e-07



In [94]:
# Визуализируем изменение функционала ошибки
plt.figure(figsize=(9,7))
plt.plot(range(len(errors)), errors)
plt.title('MSE')
plt.xlabel('Число итераций')
plt.ylabel('MSE')

NameError: name 'SGD' is not defined

<Figure size 648x504 with 0 Axes>

### __2.__ Модифицировать решение первого задания путем добавления L2 регуляризации и сравнить результаты.<br>

### __$^*$3.__ Модернизировать решение задания 2, заменив L2 регуляризацию на L1 регуляризацию.<br>
### __$^*$4.__ Построить график изменения весов от числа итераций при GD и SGD (на одном рисунке).<br>
### __$^*$5.__ Построить график изменения весов для L1 и L2 регуляризации от коэффициента регуляризации.<br>

Методичка https://colab.research.google.com/drive/1nEC_D2y-maMuvbN-5iYFAjzLUw0eGOir